In [1]:
import os

In [2]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\python_project\\DataScience\\StoresSalesPrediction\\BigMartStoreSalesPrediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    train_data_path: Path
    test_data_path: Path
    mismatch_data_type_path: Path

In [6]:
from salesPrediction.constants import *
from salesPrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])       

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            mismatch_data_type_path = config.mismatch_data_type_path
        )

        return data_transformation_config
 

In [8]:
import os
from salesPrediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
import json




In [10]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config

    def convert_column_datatype(self):
        try:

            with open(self.config.mismatch_data_type_path, "r") as f:
                mismatched_cols =  json.loads(f.read())
                print(mismatched_cols)

            data = pd.read_csv(self.config.train_data_path)
            for col, dtype_info in mismatched_cols.items():
                if dtype_info == "string":
                    data[col].astype("string")

        except Exception as e:
            raise e
    
    def train_test_splitting(self):
        # we already have train and test data set in the resources folder, we only need to copy them 
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        print(self.config.data_path)
        train_data.to_csv(os.path.join(self.config.data_path, "Train.csv"), index=False)
        test_data.to_csv(os.path.join(self.config.data_path, "Test.csv"), index= False)

        logger.info("Completed saving train and test data to transformation folder")
        logger.info(train_data.shape)
        logger.info(test_data.shape)

        print(train_data.shape)
        print(test_data.shape)

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert_column_datatype()
    # data_transformation.train_test_splitting()
except Exception as e:
    raise e


[2025-02-27 01:33:46,673: common:yaml file: config\config.yaml loaded successfully]
[2025-02-27 01:33:46,677: common:yaml file: params.yaml loaded successfully]
[2025-02-27 01:33:46,681: common:yaml file: schema.yaml loaded successfully]
[2025-02-27 01:33:46,683: common:created directory at artifacts]
[2025-02-27 01:33:46,683: common:created directory at artifacts/data_transformation]
{'Item_Identifier': {'expected': 'string', 'actual': 'object'}, 'Item_Fat_Content': {'expected': 'string', 'actual': 'object'}, 'Item_Type': {'expected': 'string', 'actual': 'object'}, 'Outlet_Identifier': {'expected': 'string', 'actual': 'object'}, 'Outlet_Size': {'expected': 'string', 'actual': 'object'}, 'Outlet_Location_Type': {'expected': 'string', 'actual': 'object'}, 'Outlet_Type': {'expected': 'string', 'actual': 'object'}}
